To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4100 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `ChatML` format for conversation style finetunes. We use [Open Assistant conversations](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style) in ShareGPT style. ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Normally one has to train `<|im_start|>` and `<|im_end|>`. We instead map `<|im_end|>` to be the EOS token, and leave `<|im_start|>` as is. This requires no additional training of additional tokens.

Note ShareGPT uses `{"from": "human", "value" : "Hi"}` and not `{"role": "user", "content" : "Hi"}`, so we use `mapping` to map it.

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [11]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset # Ensure this import is present

# 1. Load Your Custom Dataset
# Ensure your 'one_100.json' file is uploaded to your Colab environment's root directory.
# If your file is in the root of your Colab environment:
dataset = load_dataset("json", data_files="one_100.json", split="train")

# If your file is in Google Drive after mounting (uncomment and adjust path if using Drive):
# from google.colab import drive
# drive.mount('/content/drive')
# dataset = load_dataset("json", data_files="/content/drive/MyDrive/path/to/one_100.json", split="train")


# 2. Configure the Tokenizer's Chat Template
# This configures your tokenizer for the ChatML format, which your formatting function creates.
# 'tokenizer' object should be defined in a previous cell from where you load your model.
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    # The mapping below is for datasets that provide messages as a list of dictionaries (e.g., ShareGPT format).
    # Since your `formatting_prompts_func` directly builds the ChatML string, this mapping is less critical for its direct operation,
    # but it ensures the tokenizer's internal template understands the roles.
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

# 3. Your Custom Formatting Function
# This function converts your "text_input" and "output" columns into the ChatML string format.
def formatting_prompts_func(examples):
    texts = []
    for i in range(len(examples["text_input"])):
        # Construct a single conversation turn for each example in ChatML format
        full_conversation = (
            "<|im_start|>user\n"
            + examples["text_input"][i]
            + "<|im_end|>\n<|im_start|>assistant\n"
            + examples["output"][i]
            + "<|im_end|>"
        )
        texts.append(full_conversation)
    return { "text" : texts, }
pass

# 4. Apply the Formatting Function to Your Dataset
# This will create a new 'text' column in your dataset containing the formatted strings.
dataset = dataset.map(formatting_prompts_func, batched = True,)

# You can uncomment these lines to inspect the loaded and formatted dataset
# print(dataset)
# print(dataset[0]["text"])

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Let's see how the `ChatML` format works by printing the 5th element

In [12]:
print(dataset[5]["text"])

<|im_start|>user
Attack Type: INIT_DATA
Attack Explanation: This cyberattack targets smart grid systems using Function Code 15 (Initialize Data) in the DNP3 protocol to reset configuration or operational data on outstations. The attacker either impersonates a master or performs a Man-in-the-Middle (MiTM) attack to intercept and modify a legitimate packet, injecting a request that forces the outstation to reinitialize its data to default values. This unauthorized reset can result in the outstation reporting inaccurate or outdated status information, disrupting normal grid monitoring and control operations. The attack compromises data integrity and undermines the reliability of grid communications by overriding values set by legitimate master stations.

Top 5 Important Features:
  1. Flow IAT Mean (Importance: 0.723114, Value: 945734.0, Comparison: value 945734.0 above 75th percentile 944486.426 (higher than typical normal))
     → Feature Explanation: Mean time interval between packets 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [16]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4100,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.52 GB of memory reserved.


In [15]:
# Assuming 'dataset' and 'tokenizer' are already loaded from previous cells

max_length = 0
for i, example in enumerate(dataset):
    # Reconstruct the full ChatML conversation string for each example
    # This should be identical to how your formatting_prompts_func works
    full_conversation = (
        "<|im_start|>user\n"
        + example["text_input"]
        + "<|im_end|>\n<|im_start|>assistant\n"
        + example["output"]
        + "<|im_end|>"
    )

    # Tokenize the full conversation string
    # add_special_tokens=True ensures ChatML tokens are counted
    tokenized_output = tokenizer(full_conversation, add_special_tokens=True)
    current_length = len(tokenized_output["input_ids"])

    if current_length > max_length:
        max_length = current_length
        # Optional: Print the index and content of the longest example for inspection
        # print(f"New max length found at index {i}: {max_length}")
        # print(f"Example text: {full_conversation[:500]}...") # Print first 500 chars

print(f"\nCalculated maximum token length: {max_length}")

# Recommend a max_seq_length, usually a power of 2 slightly above the max_length
# Common values are 512, 1024, 2048, 4096
if max_length <= 512:
    recommended_max_seq_length = 512
elif max_length <= 1024:
    recommended_max_seq_length = 1024
elif max_length <= 2048:
    recommended_max_seq_length = 2048
elif max_length <= 4096:
    recommended_max_seq_length = 4096
else:
    recommended_max_seq_length = ((max_length // 512) + 1) * 512 # Round up to nearest 512

print(f"Recommended max_seq_length to use: {recommended_max_seq_length}")


Calculated maximum token length: 2592
Recommended max_seq_length to use: 4096


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 82 | Num Epochs = 6 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,000,000,000 (0.60% trained)


Step,Training Loss
1,0.686700
2,0.695800
3,0.609000
4,0.593900
5,0.538600
6,0.565000
7,0.493000
8,0.483200
9,0.420900
10,0.459300


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

870.2993 seconds used for training.
14.5 minutes used for training.
Peak reserved memory = 6.289 GB.
Peak reserved memory for training = 1.769 GB.
Peak reserved memory % of max memory = 42.643 %.
Peak reserved memory for training % of max memory = 11.995 %.


<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `ChatML`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [19]:
from unsloth.chat_templates import get_chat_template
# Assuming 'model' and 'tokenizer' are already loaded from your fine-tuned setup.
# Make sure you have loaded your fine-tuned LoRA adapter or merged model.

# This part configures your tokenizer's chat template. If it's already run, you don't need to rerun it unless you restarted the runtime.
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Ensures the tokenizer understands ChatML format
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style mapping
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

# Enable native 2x faster inference. This is crucial for performance.
FastLanguageModel.for_inference(model)

# Define your input text for inference. This is the content you want the model to respond to.
my_input_text = """Attack Type: WARM_RESTART

Attack Explanation:
This cyberattack targets a DNP3 outstation by instructing it to perform a warm restart, which triggers a partial restart focused on reinitializing only the DNP3 application layer, rather than the entire system. The attacker, acting as a malicious master station, sends a DNP3 packet containing the Warm Restart function code to the target.
As a result, the outstation temporarily stops responding to legitimate master requests while the application layer restarts. Although the physical device remains powered on and other system components are unaffected, the disruption in DNP3 services can delay communication, potentially impacting grid monitoring and control.
The attack follows the same approach as the Disable Unsolicited Message and Cold Restart attacks: it exploits the fact that DNP3 outstations accept commands from any master without strong authentication, making them vulnerable to unauthorized restart requests.

Top 5 Important Features:
 1. TotLen Bwd Pkts (Importance: 0.753850, Value: 2059.0, Comparison: value 2059.0 above 75th percentile 1994.0 (higher than typical normal))
    → Feature Explanation: Total size of packets in the backward direction
 2. Idle Max (Importance: 0.715813, Value: 1934724.5, Comparison: value 1934724.5 above 75th percentile 0.0 (higher than typical normal))
    → Feature Explanation: Maximum idle time before active
 3. Tot Bwd Pkts (Importance: 0.711102, Value: 69.0, Comparison: value 69.0 above normal max 65.0)
    → Feature Explanation: Total packets sent in the backward direction
 4. Fwd IAT Tot (Importance: 0.706493, Value: 118975700.0, Comparison: value 118975700.0 above normal max 118059661.0)
    → Feature Explanation: Total time between packets sent in the forward direction
 5. Bwd Pkt Len Mean (Importance: 0.693778, Value: 29.84058, Comparison: value 29.84058 within normal range (between 25th and 75th percentile))"""

# Format your input prompt for the model using the ChatML template
# This structure is crucial because your model was fine-tuned on this specific format.
messages = [
    {"role": "user", "content": my_input_text},
]

# Apply the chat template to tokenize the messages
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Essential: tells the model it's time to generate the assistant's part
    return_tensors = "pt",
).to("cuda") # Move inputs to GPU

# Generate the output from the model
outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 600, # Adjust this value based on how long you expect the model's output to be
    use_cache = True,
    pad_token_id = tokenizer.eos_token_id, # Good practice for generation
    eos_token_id = tokenizer.eos_token_id, # Good practice for generation
)

# Decode the generated tokens back into human-readable text
response = tokenizer.batch_decode(outputs)
full_decoded_text = response[0]

# Extract only the assistant's generated response
assistant_prefix = "<|im_start|>assistant\n"
try:
    assistant_output_start = full_decoded_text.rfind(assistant_prefix)
    if assistant_output_start != -1:
        generated_output = full_decoded_text[assistant_output_start + len(assistant_prefix):].strip()
        # Remove any potential trailing end-of-message token
        generated_output = generated_output.split("<|im_end|>")[0].strip()
        print("\nGenerated Output:\n", generated_output)
    else:
        print("\nCould not find assistant's response prefix in generated text.")
        print("Full Generated Text (for debugging):\n", full_decoded_text)
except Exception as e:
    print(f"Error parsing generated output: {e}")
    print("Full Generated Text (for debugging):\n", full_decoded_text)


Generated Output:
 Attack Type: WARM_RESTART

Attack Explanation:
This cyberattack targets a DNP3 outstation by instructing it to perform a warm restart, which triggers a partial restart focused on reinitializing only the DNP3 application layer, rather than the entire system. The attacker, acting as a malicious master station, sends a DNP3 packet containing the Warm Restart function code to the target.
As a result, the outstation temporarily stops responding to legitimate master requests while the application layer restarts. Although the physical device remains powered on and other system components are unaffected, the disruption in DNP3 services can delay communication, potentially impacting grid monitoring and control.
The attack follows the same approach as the Disable Unsolicited Message and Cold Restart attacks: it exploits the fact that DNP3 outstations accept commands from any master without strong authentication, making them vulnerable to unauthorized restart requests.

The to

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [22]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer # Ensure TextStreamer is imported

# Assume 'model' and 'tokenizer' are already loaded from your fine-tuned setup.
# Make sure you have loaded your fine-tuned LoRA adapter or merged model.

# This part configures your tokenizer's chat template. If it's already run, you don't need to rerun it.
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    map_eos_token = True,
)

# Enable native 2x faster inference. This is crucial for performance.
FastLanguageModel.for_inference(model)

# Define your input text for inference.
my_input_text = """Attack Type: WARM_RESTART

Attack Explanation:
This cyberattack targets a DNP3 outstation by instructing it to perform a warm restart, which triggers a partial restart focused on reinitializing only the DNP3 application layer, rather than the entire system. The attacker, acting as a malicious master station, sends a DNP3 packet containing the Warm Restart function code to the target.
As a result, the outstation temporarily stops responding to legitimate master requests while the application layer restarts. Although the physical device remains powered on and other system components are unaffected, the disruption in DNP3 services can delay communication, potentially impacting grid monitoring and control.
The attack follows the same approach as the Disable Unsolicited Message and Cold Restart attacks: it exploits the fact that DNP3 outstations accept commands from any master without strong authentication, making them vulnerable to unauthorized restart requests.

Top 5 Important Features:
 1. TotLen Bwd Pkts (Importance: 0.753850, Value: 2059.0, Comparison: value 2059.0 above 75th percentile 1994.0 (higher than typical normal))
    → Feature Explanation: Total size of packets in the backward direction
 2. Idle Max (Importance: 0.715813, Value: 1934724.5, Comparison: value 1934724.5 above 75th percentile 0.0 (higher than typical normal))
    → Feature Explanation: Maximum idle time before active
 3. Tot Bwd Pkts (Importance: 0.711102, Value: 69.0, Comparison: value 69.0 above normal max 65.0)
    → Feature Explanation: Total packets sent in the backward direction
 4. Fwd IAT Tot (Importance: 0.706493, Value: 118975700.0, Comparison: value 118975700.0 above normal max 118059661.0)
    → Feature Explanation: Total time between packets sent in the forward direction
 5. Bwd Pkt Len Mean (Importance: 0.693778, Value: 29.84058, Comparison: value 29.84058 within normal range (between 25th and 75th percentile))"""

# Format your input prompt for the model using the ChatML template
messages = [
    {"role": "user", "content": my_input_text},
]

# Apply the chat template to tokenize the messages
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Essential: tells the model it's time to generate the assistant's part
    return_tensors = "pt",
).to("cuda") # Move inputs to GPU

# Initialize the TextStreamer
text_streamer = TextStreamer(tokenizer, skip_special_tokens = True)

# Generate the output using the streamer
# Note: When using a streamer, the `model.generate` call itself will print the output.
# You typically don't store the `outputs` variable if you're just streaming to console.
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 700, # Use the recommended value from our previous discussion
    use_cache = True,
    pad_token_id = tokenizer.eos_token_id,
    eos_token_id = tokenizer.eos_token_id,
)

<|im_start|>user
Attack Type: WARM_RESTART

Attack Explanation:
This cyberattack targets a DNP3 outstation by instructing it to perform a warm restart, which triggers a partial restart focused on reinitializing only the DNP3 application layer, rather than the entire system. The attacker, acting as a malicious master station, sends a DNP3 packet containing the Warm Restart function code to the target.
As a result, the outstation temporarily stops responding to legitimate master requests while the application layer restarts. Although the physical device remains powered on and other system components are unaffected, the disruption in DNP3 services can delay communication, potentially impacting grid monitoring and control.
The attack follows the same approach as the Disable Unsolicited Message and Cold Restart attacks: it exploits the fact that DNP3 outstations accept commands from any master without strong authentication, making them vulnerable to unauthorized restart requests.

Top 5 Imp

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|im_start|>user
What is a famous tall tower in Paris?<|im_end|> 
<|im_start|>assistant
The Eiffel Tower is a famous tall tower in Paris. It is one of the most recognizable landmarks in the world and is a popular tourist destination. The tower was built in 1889 for the World's Fair and is named after Gustave Eiffel, the engineer who designed and built it. The tower stands at a height of 324 meters (1,063 feet) and is made of iron. It is located on the Champ de Mars, a large public park in Paris.<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoModelForPeftCausalLM
    from transformers import AutoTokenizer

    model = AutoModelForPeftCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
